In [ ]:
from selenium import webdriver 
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

# create Chromeoptions instance 
options = webdriver.ChromeOptions() 
 
# adding argument to disable the AutomationControlled flag 
options.add_argument("--disable-blink-features=AutomationControlled") 
 
# exclude the collection of enable-automation switches 
options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
 
# turn-off userAutomationExtension 
options.add_experimental_option("useAutomationExtension", False) 
 
# setting the driver path and requesting a page 
driver = webdriver.Chrome(options=options) 
 
# changing the property of the navigator value for webdriver to undefined 
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 
 
driver.get("https://www.shopee.vn")
time.sleep(3.5)
# wait = WebDriverWait(driver, 3)
# element = wait.until(
#     EC.presence_of_element_located(
#         (By.TAG_NAME, "body")
#     )
# )
# close the driver
# driver.close()

In [ ]:
# load_cookies_example.py
import json
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

CHROMEDRIVER_PATH = "chromedriver"
COOKIES_FILE = "cookies.json"
BASE_URL = "https://shopee.vn"

def init_driver(chromedriver_path=CHROMEDRIVER_PATH, headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless=new")
        options.add_argument("--start-maximized")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option("useAutomationExtension", False)
        # service = Service(chromedriver_path)
        driver = webdriver.Chrome(options=options)
    return driver

def load_cookies(driver, path=COOKIES_FILE, base_url=BASE_URL):
    with open(path, "r", encoding="utf-8") as f:
        cookies = json.load(f)

    driver.get(base_url)  # phải vào 1 page cùng domain trước khi add cookie
    time.sleep(1)  # cho page load sơ
    added = 0
    for c in cookies:
        cookie = c.copy()
        # Selenium add_cookie không chấp nhận domain dạng .example.com khi hiện tại driver.get() đang ở example.com
        # Remove fields Selenium không cần hoặc có thể gây lỗi
        cookie.pop("sameSite", None)   # optional
        if "expiry" in cookie:
            # ensure expiry is int
            try:
                cookie["expiry"] = int(cookie["expiry"])
            except Exception:
                cookie.pop("expiry", None)
        try:
            driver.add_cookie(cookie)
            added += 1
        except Exception as e:
            # một số cookie có flag secure/httponly hay domain không khớp sẽ fail
            print("Could not add cookie:", cookie.get("name"), "->", e)
    print(f"Added {added}/{len(cookies)} cookies")
    driver.refresh()
    time.sleep(2)

if __name__ == "__main__":
    driver = init_driver(headless=False)
    try:
        load_cookies(driver, COOKIES_FILE, BASE_URL)
        # kiểm tra đã login chưa: ví dụ tìm avatar / tên người dùng / element logout
        # thay selector cho phù hợp
        try:
            avatar = driver.find_element("css selector", ".user-avatar, ._1F7bJA")  # ví dụ
            if avatar.is_displayed():
                print("Có vẻ đã login (avatar tìm thấy).")
        except Exception:
            print("Không tìm thấy element chứng tỏ login. Có thể cookie hết hạn hoặc selector không chính xác.")
    finally:
        # giữ cửa sổ 10s để bạn nhìn kết quả
        time.sleep(10)
        driver.quit()


In [ ]:
import requests, json

def check_cookie_valid(cookies_json_path):
    with open(cookies_json_path, "r", encoding="utf-8") as f:
        cookies = json.load(f)
    s = requests.Session()
    s.headers.update({"User-Agent": "Mozilla/5.0"})
    for c in cookies:
        domain = c.get("domain","")
        if domain.startswith("."): domain = domain[1:]
        s.cookies.set(c["name"], c["value"], domain=domain, path=c.get("path","/"))
    # Thử gọi API nội bộ user info nếu bạn tìm được (khuyên dùng)
    resp = s.get("https://shopee.vn/api/v4/account/get_info", timeout=10)
    print(resp.status_code)
    try:
        print(resp.json())
    except:
        print("Response text length:", len(resp.text))
    # Fallback: check homepage HTML for login text
    r2 = s.get("https://shopee.vn", timeout=10)
    return "Đăng xuất" in r2.text or "Tài khoản" in r2.text

print(check_cookie_valid("cookies.json"))


In [ ]:
from datetime import datetime
with open("cookies.json", "r", encoding="utf-8") as f:
    cookies = json.load(f)
expiry_timestamp = cookies[0]['expiry']
expiry_date = datetime.fromtimestamp(expiry_timestamp)
current_time = datetime.now()
print(f"Thời gian hết hạn của cookie: {expiry_date}")
print(f"Thời gian hiện tại:          {current_time}")


In [ ]:
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
options = Options()
options.add_argument("--headless=new")
options.add_argument("--start-maximized")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
# service = Service(chromedriver_path)
driver = webdriver.Chrome(options=options)
driver.get("https://shopee.vn")

try:
    with open("cookies.json", "r") as f:
        cookies = json.load(f)

    for cookie in cookies:
        if 'expiry' in cookie:
            cookie['expiry'] = int(cookie['expiry'])
        
        if 'sameSite' in cookie:
            del cookie['sameSite']
        
        driver.add_cookie(cookie)
    
    print("Cookies đã được tải thành công!")

except FileNotFoundError:
    print("Lỗi: Không tìm thấy file cookies.json.")
except Exception as e:
    print(f"Có lỗi xảy ra khi thêm cookies: {e}")

# Tải lại trang để cookies có hiệu lực
driver.refresh()
print("Đã tải lại trang với cookies. Bây giờ bạn có thể tương tác với trang web.")
time.sleep(5)  # Chờ trang tải hoàn toàn
html_content = driver.page_source
soup = BeautifulSoup(html_content, 'html.parser')
t = soup.find('h1')
print(t)
# Để đảm bảo trình duyệt không tự động đóng, bạn có thể thêm một dòng input()
input("Nhấn Enter để đóng trình duyệt...")
driver.quit()


In [ ]:
import requests
cookies = {
    "_ga": "GA1.1.1719619335.1694848832",
    "_fbp": "fb.1.1739512332184.927524205294540691",
    "SPC_U": "380256125",
    "SPC_T_IV": "QXdWVklyaU01aXoyVVVUUQ==",
    "SPC_T_ID": "V0riExw6e7DMh19BYIPnSU6s9uFOXyoXIPLqDybVYksAGVLPyVfCimQZvVkzmoT0KyBA/R1H+2m4mz/skA4UKWTMSWpElcrm0VUIEQrQnh3XobnQXx3dxJa7pEifTIZlfUXZ5saUY9/denuM7fy3QB8G72su5+2DXxJ8iBFNuwo=",
    "SPC_ST": ".N3NGcTlyYnE4TG85VUdOeehah89C0l24KbK8ZH1FqPSKNlm9nAWqY192gyhxbB+Lv0yuZ1bZXnwIhdmCrRSCtGAdr0IF8sgZD2gN8wJlPMWt283//DE+T6TLGP/ZfjeqtX1KeoJLz46g5je4SFUW0X/UkT8oi5BtK82sHAnwzhzwFtGGqxDFrmA87AKc6mQKnccHdmW30NVPpEuhYC4GAUyC+e/4LS3dUA1e97bOAJWGaCGZRpN91GhyL/jAPzUf",
    "SPC_SI": "E1fKaAAAAABRUkREOWQ1ZxiAvAAAAAAAUXBObVNieW0=",
    "SPC_SEC_SI": "v1-TTRodjJLZ3pTUmpqbnFBccZBF4nXhgxyU+6O6KCxCnySX9arvyycsEO2UjFL3zZ/LKpeZg99ixhFXsM09/Mmq3LdmOpngr4hkvwUJ1uxEZQ=",
    "SPC_IA": "1",
    "SPC_CLIENTID": "lUEGmIsZZ53MtazDyaguxiupqoljzvkm"
}

url = "https://shopee.vn/search?keyword=shirt"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Referer": "https://shopee.vn/"
}

response = requests.get(url, headers=headers, cookies=cookies)

data = response.text

print(data)